## Preprocessing

In [ ]:
# pip install tensorflow

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df = application_df[application_df.STATUS == 1]

In [3]:
comp_name_df = application_df[['EIN','NAME']]
comp_name_df.head()

,EIN,NAME
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN','NAME', 'STATUS','SPECIAL_CONSIDERATIONS'])

In [5]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [6]:
# Determine the number of unique values in each column.
cat_name_list = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[cat_name_list].nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
unique_values = application_df['APPLICATION_TYPE'].value_counts()
unique_values

T3     27032
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = unique_values[unique_values < 200].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27032
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
class_value_counts = application_df['CLASSIFICATION'].value_counts()
class_value_counts

C1000    17323
C2000     6073
C1200     4837
C3000     1918
C2100     1882
         ...  
C1728        1
C3700        1
C1732        1
C4120        1
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_value_counts[class_value_counts < 200].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17323
C2000     6073
C1200     4837
C3000     1918
C2100     1882
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [11]:
application_df[cat_name_list].nunique()

APPLICATION_TYPE    9
AFFILIATION         6
CLASSIFICATION      8
USE_CASE            5
ORGANIZATION        4
INCOME_AMT          9
dtype: int64

In [12]:
# # Create a OneHotEncoder instance to convert categorical data to numeric
enc = OneHotEncoder(sparse = False)

In [13]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[cat_name_list]))

encode_df.columns = enc.get_feature_names(input_features = cat_name_list)

encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# merge back to orginal df and drop original unencoded columns
encoded_charity_df  = application_df.merge(encode_df, left_index = True, right_index = True)\
                                .drop(columns = cat_name_list)

encoded_charity_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Split our preprocessed data into our features and target arrays
y = encoded_charity_df['IS_SUCCESSFUL'].values
X = encoded_charity_df.drop(columns = ['IS_SUCCESSFUL']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)


In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train_scaled[0])
num_first = len(X_train_scaled[0])*2
num_second = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=num_first, input_dim = num_input, activation ='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = num_second, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 84)                3612      
                                                                 
 dense_1 (Dense)             (None, 50)                4250      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7,913
Trainable params: 7,913
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 749us/step - loss: 0.6861 - accuracy: 0.5605
Epoch 2/200
804/804 [==============================] - 1s 751us/step - loss: 0.6808 - accuracy: 0.5701
Epoch 3/200
804/804 [==============================] - 1s 712us/step - loss: 0.6795 - accuracy: 0.5742
Epoch 4/200
804/804 [==============================] - 1s 723us/step - loss: 0.6790 - accuracy: 0.5756
Epoch 5/200
804/804 [==============================] - 1s 717us/step - loss: 0.6783 - accuracy: 0.5765
Epoch 6/200
804/804 [==============================] - 1s 809us/step - loss: 0.6779 - accuracy: 0.5770
Epoch 7/200
804/804 [==============================] - 1s 739us/step - loss: 0.6779 - accuracy: 0.5745
Epoch 8/200
804/804 [==============================] - 1s 722us/step - loss: 0.6774 - accuracy: 0.5787
Epoch 9/200
804/804 [==============================] - 1s 734us/step - loss: 0.6772 - accuracy: 0.5774
Epoch 10/200
804/804 [==============================] - 1s 773us/step - l

804/804 [==============================] - 1s 844us/step - loss: 0.6640 - accuracy: 0.5895
Epoch 80/200
804/804 [==============================] - 1s 853us/step - loss: 0.6644 - accuracy: 0.5896
Epoch 81/200
804/804 [==============================] - 1s 857us/step - loss: 0.6646 - accuracy: 0.5902
Epoch 82/200
804/804 [==============================] - 1s 847us/step - loss: 0.6645 - accuracy: 0.5904
Epoch 83/200
804/804 [==============================] - 1s 833us/step - loss: 0.6654 - accuracy: 0.5898
Epoch 84/200
804/804 [==============================] - 1s 832us/step - loss: 0.6645 - accuracy: 0.5897
Epoch 85/200
804/804 [==============================] - 1s 834us/step - loss: 0.6640 - accuracy: 0.5917
Epoch 86/200
804/804 [==============================] - 1s 862us/step - loss: 0.6641 - accuracy: 0.5895
Epoch 87/200
804/804 [==============================] - 1s 859us/step - loss: 0.6641 - accuracy: 0.5909
Epoch 88/200
804/804 [==============================] - 1s 850us/step - loss:

804/804 [==============================] - 1s 1ms/step - loss: 0.6606 - accuracy: 0.5936
Epoch 159/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6605 - accuracy: 0.5919
Epoch 160/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6609 - accuracy: 0.5931
Epoch 161/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6616 - accuracy: 0.5922
Epoch 162/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6605 - accuracy: 0.5934
Epoch 163/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6610 - accuracy: 0.5918
Epoch 164/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6609 - accuracy: 0.5929
Epoch 165/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6609 - accuracy: 0.5928
Epoch 166/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6613 - accuracy: 0.5929
Epoch 167/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6617 - a

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7561 - accuracy: 0.5685 - 275ms/epoch - 1ms/step
Loss: 0.7560544013977051, Accuracy: 0.5685291290283203


In [24]:
# Export our model to HDF5 file
nn.save_weights('my_model.hdf5')